pip install --upgrade tensorflow
pip install opencv-python-headless

# Resimlerin okunması, resim ve etiket dizilerinin oluşturulması aşaması

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
import numpy as np
import random
import math
import csv
import cv2
import os

2023-10-06 00:32:24.084303: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


inputBasePath içerisinde her bir sınıf için o sınıf adıyla oluşturulmuş bir klasör vardır ve her klasör içerisinde o sınıfa ait resimler yer almaktadır. bu resimler her analiz adımında yeniden okunup işlenebilir ancak bu okuma sürecini yeniden yeniden yapmamak için resimler bir kereye mahsus okunup, istenirse yeniden boyutlandırılıp, istenirse filtre uygulanıp vs. daha sonra bir numpy array olarak kaydedilir. daha sonraki analiz işlemlerinden direkt bu array okunarak hızlıca işlem yapılabilir. oluştutulan array'ler outputBasePath yoluna kaydedilecek.

In [2]:
inputBasePath    = r"/home/yildirim/Jupiter/images"
outputBasePath =  r"/home/yildirim/Jupiter/imagearrays" #bu klasörlerin daha önce oluşturulmuş olması gerek



resimler yeniden boyutlandırılmak istenirse genişlik ve yükseklik değerleri burada tanımlanıyor. özellikle state-of-art modeller resimleri belli ölçülerde daha iyi işliyor. ideal değerleri öğrenip ona göre boyut verilebilir.

In [3]:
image_width = 224
image_height = 224 

sınıf adlarını tutan bir dizi tanımlanıyor. bu sınıf adları aynı zamanda inputBasePath'te yer alan klasör isimleri


In [4]:
classes = ['cat','dog']

In [5]:
os.chdir(inputBasePath) #chdir -> change directory, inputBasePath yoluyla verilen dizine git

X = [] # resimleri yani girdileri yani X değerlerini tutmak için dizi
Y = [] # etiketleri yani Y değerlerini tutmak için dizi. her bir resmin etiketi içinde yer aldığı klasörün adı zaten

i = 0
for class1 in classes:
  os.chdir(class1) #base yoldan sonra sıradaki sınıfı gösteren klasöre konumlan
  print('=> '+class1) #o an üzerinde bulunulan sınıfı (klasör adını) yaz
  for files in os.listdir('./'): # nokta mevcut dizini gösteriyor. ./ mevcut dizin altındakiler
    img = cv2.imread(files) #dosya yolundan resmi binary array olarak okuma.resmi grayscale almak için ikinci parametreye 0 yazılır cv2.imread(files,0)
    img = cv2.resize(img, (image_width,image_height)) #isteğe bağlı olarak resize edilebilir
    X.append(img) #resmi oluşturan bit dizisini X'e ekle
    Y.append(class1) # bu resmin sınıfı içinde bulunduğu klasör adı. resmin etiketi olarak bunu Y'ye ekle
    i = i + 1
  os.chdir('..') #bir üst dizine çık. bu sınıfla ve bunu içeren klasörler işimiz bitti
  
print("X : ",len(X))
print("Y : ",len(Y))

X = np.array(X).reshape(-1,image_width,image_height,3) #-1 ile verilen ilk değerin yerinde toplam resim adedi var; 
                #bu aynı kalacak. diğer parametreler verilen width ve height'e göre ve resmin renkli olduğunu 
                #belirten 3 ile yeniden şekillendirilecek
Y = np.array(Y) #etiket adlarını içeren Y'yi reshape etmeye gerek yok. 

print("X : ",X.shape)
print("Y : ",Y.shape)

print("X : ",len(X))
print("Y : ",len(Y))

os.chdir('..') #bir üst dizine daha çıkıp sonra imagearrays klasörüne gidersek zaten outputBasePath'e ulaşmış olacağız
os.chdir("imagearrays")
# üstteki iki satır yerine bunu direkt chdir(outputBasePath) olarak da yapabilirdik
np.save(str(image_width)+'x'+str(image_height)+'_images', X) #diziyi kaydederken dosya adını en x boy_images olarak adlandır.
                                                            #'224x224_images' gibi
np.save(str(image_width)+'x'+str(image_height)+'_labels', Y) #diziyi kaydederken dosya adını en x boy_labels olarak adlandır.

print("[ INFO - STAGE1 ]  NUMPY ARRAY CREATION COMPLETED \n ")

=> cat
=> dog
X :  244
Y :  244
X :  (244, 224, 224, 3)
Y :  (244,)
X :  244
Y :  244
[ INFO - STAGE1 ]  NUMPY ARRAY CREATION COMPLETED 
 



## Sınıflandırma işlemleri

Bu aşamadan sonra daha önce oluşturulan array'ler okunarak işlem yapılacak


In [6]:
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import MaxPool2D # üstteki MaxPooling2D ile aynı şey. ister onu ister bunu kullan. 
#kodda ikisi de kullanıldığı için eklendi. yoksa biri ile yapılsaydı da olurdu.
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD



data önce numpy array olarak kaydedilen görüntüleri (data) ve sınıf (label) etiketlerini oku

In [7]:
data = np.load(r"/home/yildirim/Jupiter/imagearrays/224x224_images.npy")
labels = np.load(r"/home/yildirim/Jupiter/imagearrays/224x224_labels.npy")
data.shape

(244, 224, 224, 3)

In [8]:
labels

array(['cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat',
       'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat',
       'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat',
       'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat',
       'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat',
       'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat',
       'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat',
       'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat',
       'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat',
       'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat',
       'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat',
       'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat',
       'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat', 'cat',
       'cat', 'cat', 'cat', 'cat', 'dog', 'dog', 'dog', 'dog', 'dog',
       'dog', 'dog',


## Label Encoding

array den okunan etiketler orjinal halde, string şeklinde. bunları 0 1 2 şeklinde kodlayacağız yani label encoding yapacağız


In [9]:
labelEn = LabelEncoder() #string olan etiketleri 0 1 2 şeklinde kodla
labels = labelEn.fit_transform(labels)
labels = to_categorical(labels)

In [10]:
labels

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.

Train - Test Split

In [11]:


#veri array'e kaydedilmeden önce reshape edildiğinden array den okununca da düzgün gelir. yeniden reshape etmeye gerek yok
#eğer array'e atarken son değer 3 olarak yazılmasaydı burada reshape gerekirdi

#data =  data.reshape(-1,image_width , image_height , 3) 
                                                        



# train -test split
#%20 test %80 eğitim seti olacak şekilde böl
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size = .20, shuffle = True)


print(
"""
x_train shape: {}
x_test shape: {}
y_train shape: {}
y_test shape: {}

""".format(x_train.shape, x_test.shape, y_train.shape, y_test.shape))




x_train shape: (195, 224, 224, 3)
x_test shape: (49, 224, 224, 3)
y_train shape: (195, 2)
y_test shape: (49, 2)





## Normalizasyon (bu adım opsiyonel)

Veriler normalize edilerek piksel değer aralıkları 0-1 aralığına çekilip daha hızlı işlem yapılması sağlanabilir. Normalizasyon işlem hızını arttırır ama her görüntü için başarı artışı getirmeyebilir, belki başarıyı düşürebilir.


In [12]:
x_train_mean = np.mean(x_train)
x_train_std = np.std(x_train)

x_test_mean = np.mean(x_test)
x_test_std = np.std(x_test)

x_train = (x_train - x_train_mean)/x_train_std
x_test = (x_test - x_test_mean)/x_test_std




### Train - Validation Split

%20 test - %10 validation seti olacak şekilde ayır

validation datası, modelin eğitimi esnasında train verisini doğrulamak için yani eğitim işlemi esnasındaki test sürecini gerçekleştirmek için kullanılıyor. x_test ve y_test ise eğitim süreci bittikten sonra eğitilen modeli daha önce hiç bilmediği verilerle test etmek için


In [13]:
x_train, x_validate, y_train, y_validate = train_test_split(x_train, y_train, test_size = .10, shuffle = True,random_state=42)


### Model tanımlama

kendimize göre bir model oluşturan ve bunu geri döndüren bir fonksiyon yazalım


In [14]:
def model1(input_shape=(image_width ,image_height ,3), num_classes = 2): #parametrelerin varsayılan değerleri var. 
    #modelin giriş shape'i ve class sayısı= 3                                                                     
    #burada oluşturulan model VGG16 mimarisi aslında. değiştirilebilir.
	model = Sequential()
	chanDim = -1

	model.add(Conv2D(64, (3,3), padding="same",input_shape=input_shape))
	model.add(Activation("relu"))
	model.add(BatchNormalization(axis=chanDim))
	model.add(Conv2D(64, (3,3), padding="same"))
	model.add(Activation("relu"))
	model.add(BatchNormalization(axis=chanDim))
	model.add(MaxPooling2D(pool_size=(2, 2)))

	# 2.Layer (CONV => RELU => CONV => RELU) * 2 => POOL
	model.add(Conv2D(128, (3,3), padding="same"))
	model.add(Activation("relu"))
	model.add(BatchNormalization(axis=chanDim))
	model.add(Conv2D(128, (3,3), padding="same"))
	model.add(Activation("relu"))
	model.add(BatchNormalization(axis=chanDim))
	model.add(MaxPooling2D(pool_size=(2, 2)))

	# 3.Layer (CONV => RELU => CONV => RELU) * 2 => POOL
	model.add(Conv2D(256, (3,3), padding="same"))
	model.add(Activation("relu"))
	model.add(BatchNormalization(axis=chanDim))
	model.add(Conv2D(256, (3,3), padding="same"))
	model.add(Activation("relu"))
	model.add(BatchNormalization(axis=chanDim))
	model.add(Conv2D(256, (3,3), padding="same"))
	model.add(Activation("relu"))
	model.add(BatchNormalization(axis=chanDim))
	model.add(MaxPooling2D(pool_size=(2, 2)))

	# 4.Layer (CONV => RELU => CONV => RELU) * 2 => POOL
	model.add(Conv2D(512, (3,3), padding="same"))
	model.add(Activation("relu"))
	model.add(BatchNormalization(axis=chanDim))
	model.add(Conv2D(512, (3,3), padding="same"))
	model.add(Activation("relu"))
	model.add(BatchNormalization(axis=chanDim))
	model.add(Conv2D(512, (3,3), padding="same"))
	model.add(Activation("relu"))
	model.add(BatchNormalization(axis=chanDim))
	model.add(MaxPooling2D(pool_size=(2, 2)))

	# 5.Layer (CONV => RELU => CONV => RELU) * 2 => POOL
	model.add(Conv2D(512, (3,3), padding="same"))
	model.add(Activation("relu"))
	model.add(BatchNormalization(axis=chanDim))
	model.add(Conv2D(512, (3,3), padding="same"))
	model.add(Activation("relu"))
	model.add(BatchNormalization(axis=chanDim))
	model.add(Conv2D(512, (3,3), padding="same"))
	model.add(Activation("relu"))
	model.add(BatchNormalization(axis=chanDim))
	model.add(MaxPooling2D(pool_size=(2, 2)))

	# 1. TAM BAĞLANTI KATMANI 
	model.add(Flatten())
	model.add(Dense(4096))
	model.add(Activation("relu"))
	model.add(BatchNormalization())
	model.add(Dropout(0.5))

	# 2. TAM BAĞLANTI KATMANI 
	model.add(Flatten())
	model.add(Dense(4096))
	model.add(Activation("relu"))
	model.add(BatchNormalization())
	model.add(Dropout(0.5))

	# SOFTMAX
	model.add(Dense(num_classes))
	model.add(Activation("softmax"))

	return model

In [15]:
# bir başka model
def model2(input_shape=(image_width ,image_height ,3), num_classes = 2):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',padding = 'Same',input_shape=input_shape))
    model.add(Conv2D(64,kernel_size=(3, 3), activation='relu',padding = 'Same',))
    model.add(MaxPool2D(pool_size = (2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(128, (3, 3), activation='relu',padding = 'Same'))
    model.add(Conv2D(64, (3, 3), activation='relu',padding = 'Same'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax')) #ikili sınıflama olsaydı sigmoid kullanılırdı. 
                            #bu durumda zaten num_classes 1 olurdu. yani çıkış nöronu 1 tane olurdu.
    return model

In [16]:
model = model2()
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 112, 112, 64)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 64)      73792     
                                                        

2023-10-06 00:32:26.471516: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.



Modeli görselleştirme

In [17]:
pip install pydot #görselleştirme için gerekli kütüphane

Note: you may need to restart the kernel to use updated packages.


In [18]:
tf.keras.utils.plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.



Optimizer tanımla

Eğitim işlemi sonucunda ağın bulduğu sonuç ile gerçekte olması gereken sonuç arasındaki fark ile oluşan hatayı loss function ile hesaplıyoruz. Loss fonksiyonu çoklu sınıflamalarda "categorical_crossentropy" , ikili sınıflamada "binary_crossrntropy" olarak seçiliyor. Loss hesaplandıktan sonra geriye yayılımla tüm parametrelerin optimize edilmesi gerekiyor. Bu aşamada kullanılan adım uzunluğu "learning rate" in (LR) duruma göre adaptif bir şekilde değiştirilmesi sonucu daha verimli kılar. "Learning rate" çok küçük olursa işlem uzun sürer, çok büyük olursa hatanın minimum değeri kaçırılabilir. Bu nedenle LR optimize edilmelidir. Bu iş için adaptif momentum optimizer yani "adam" optimizer kullanacağız.


In [19]:
optimizer = Adam(learning_rate=0.0001)


LR annealer tanımlama

Eğitim esnasında monitör edilen parametrede ilerleme olmuyorsa LR değiştirilir. Bu işlemi Keras.callbacks içindeki ReduceLROnPlateau fonksiyonu ile yapabiliriz.


In [20]:
# Set a learning rate annealer
from tensorflow.keras.callbacks import ReduceLROnPlateau
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience=3, verbose=1,  factor=0.5, min_lr=0.00001)



modeli derle

Loss fonksiyonu çoklu sınıflamalarda "categorical_crossentropy" , ikili sınıflamada "binary_crossentropy" olarak seçilir. burada üç sınıf olduğu için categorical_crossentropy. çoklu sınıflamada modelin sonundaki sınıflayıcıda (flatten'den sonraki kısım) çıkışta sınıf sayısı kadar nöron olur.

İkili sınıflama yapılıyorsa çıkış nöron sayısı tektir. bu tek çıkış 0 veya 1 olarak bir değer verir. bu durumda loss fonksiyonu binary_crossentropy olur.

DİKKAT: iki sınıfllı bir veri var/yok şeklinde ise ikili sınıflama uygundur. ama kırmızı-siyah, elma-çilek vs. gibi (yani kırmızı kırmızı değil şeklinde değil veya elma elma değil şeklinde yani var yok halinde değil) ve bu sınıflar 1, 2 gibi kodlanmışsa, çıkışta yine 2 nöron olur ve sınıflamada kategoriktir, binary değildir. bu durum aslında yapılan kurgu ile ilgili.


In [21]:
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])


epoch ve batch size tanımla

epoch = model kaç iterasyon çalışacak batch size = resimler modele bit dizisi matrisleri olarak alınır. her bir adımda kaç resmin bit dizisi alınacak, yani kaç resim alınacak. bir seferde alınan resimler yığın (batch), yığındaki resim sayısı batch size

cost fonksiyonu her bir batch için hesaplanır, buna göre geri yayılım yapılır. her bir resim için yapılsaydı süreç uzardı (belki hesap daha hassas olurdu). batch size azaldıkça daha ince hesap yapılır ama süreç uzar, batch size artarsa cost hesabı daha üstünkörü olur, başarı düşebilir. bu nedenle batch size optimum şekilde seçilmeli.


In [22]:
epc = 5 
bs = 8


modeli çalıştır

modeli fit edince çalışır. modelin her bir aşamasındaki sonuçlar history değişkenine raporlanır.


In [ ]:
history = model.fit(x_train,y_train, batch_size=bs,
                              epochs = epc, validation_data = (x_validate,y_validate),
                              verbose = 1, callbacks=[learning_rate_reduction])

Epoch 1/5
22/22 [==============================] - 35s 2s/step - loss: 0.8684 - accuracy: 0.4857 - val_loss: 0.7030 - val_accuracy: 0.4500 - lr: 1.0000e-04
Epoch 2/5
22/22 [==============================] - 37s 2s/step - loss: 0.6961 - accuracy: 0.5314 - val_loss: 0.6946 - val_accuracy: 0.4500 - lr: 1.0000e-04
Epoch 3/5
22/22 [==============================] - 31s 1s/step - loss: 0.6770 - accuracy: 0.6400 - val_loss: 0.6919 - val_accuracy: 0.5500 - lr: 1.0000e-04
Epoch 4/5
 1/22 [>.............................] - ETA: 31s - loss: 0.6098 - accuracy: 0.8750

In [ ]:
# history nin içindeki history değerinin anahtarları raporu alınabilecek değerlerin adlarını gösterir
history.history.keys()


Doğruluk grafiklerini çiz

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


Hata grafiklerini çiz


In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')  

In [ ]:
Y_pred = model.predict(x_test)   
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(y_test,axis = 1)  #test süreci için Y_true = np.argmax(y_test,axis = 1)
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)
# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes = range(3))

In [ ]:
from sklearn.metrics import classification_report
report = classification_report(Y_true, Y_pred_classes)
print(report)



İstenirse eğitilen model daha sonra kullanılmak üzere, hesaplanan ağırlıkları ile kaydedilebilir. bunun için de bir yol tanımlaması yapabiliriz.


In [ ]:
os.chdir(r"/home/yildirim/Jupiter/models") #modeli kaydetmek için 
model.save('model1.h5')